In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

import os

In [10]:
def extract_data(dirname='data/blogs/'):
    
    gender = []
    age = []
    occupation = []
    zodiac = []
    
    for filename in os.listdir(dirname):
        filename_tokens = filename.split('.')
        
        gender.append(filename_tokens[1])
        age.append(int(filename_tokens[2]))
        occupation.append(filename_tokens[3])
        zodiac.append(filename_tokens[4])
    
    return (gender, age, occupation, zodiac)

In [11]:
gender, age, occupation, zodiac = extract_data()

In [23]:

def get_content(data_dir='data/blogs/'):
    blog_posts_text = []
    
    for filename in os.listdir(data_dir):
        full_filename = data_dir + filename
        
        with open(full_filename) as infile:
            parsed_xml = BeautifulSoup(infile, 'lxml')
            
            posts = parsed_xml.find_all('post')
            post_text = ''
            
            for post in posts:
                post = post.strip()
                post = post.lower()
                post_text += post
            
            blog_posts_text.append(post_text)
    
    return blog_posts_text

In [24]:
blog_posts_text = get_content()

TypeError: 'NoneType' object is not callable